In [37]:

import os 
import matplotlib.pylab as plt
import pandas as pd

import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential,Model
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [38]:
train_values = pd.read_csv('/Users/ShaimaShoukat/Desktop/BDProject/train_values.csv', index_col='building_id')
train_labels = pd.read_csv('/Users/ShaimaShoukat/Desktop/BDProject/train_labels.csv', index_col='building_id')
test_values = pd.read_csv('/Users/ShaimaShoukat/Desktop/BDProject/test_values.csv', index_col='building_id')

#Missing values
#dataset = pd.concat(objs=[train_values, test_values], axis=0)

train_values = train_values.drop(['geo_level_2_id', 'geo_level_3_id'], axis=1)
#test_values = test_values.drop(['geo_level_2_id', 'geo_level_3_id'], axis=1)

In [39]:
train_values['geo_level_1_id']= train_values['geo_level_1_id'].astype('object')
test_values['geo_level_1_id']= test_values['geo_level_1_id'].astype('object')

# Pre-Processing 

In [4]:
#Converting categorical variables to numerical using Label Encoder
from sklearn.preprocessing import LabelEncoder
#columns=['foundation_type','land_surface_condition','roof_type','ground_floor_type','other_floor_type','position','plan_configuration','legal_ownership_status']
X_cat = train_values.copy()
X_cat = train_values.select_dtypes(include=['object'])
X_enc = X_cat.copy()
labelencoder = LabelEncoder()
X_enc = X_enc.apply(LabelEncoder().fit_transform) #
train_values = train_values.drop(X_cat.columns, axis=1)
train_values = pd.concat([train_values,X_enc], axis=1)


# Over Sampling

In [5]:
#over sampling to deal with class imbalance
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')
train_values_sm,train_labels_sm = smote.fit_sample(train_values,train_labels.values.ravel())
X_resampled = pd.DataFrame(train_values_sm, columns=train_values.columns)
Y_resampled = pd.DataFrame(train_labels_sm, columns=train_labels.columns)

In [6]:
#Changing the type of categorical variables to 'object'
X_resampled['geo_level_1_id']= X_resampled['geo_level_1_id'].astype('object')
X_resampled['foundation_type']= X_resampled['foundation_type'].astype('object')
X_resampled['land_surface_condition']= X_resampled['land_surface_condition'].astype('object')
X_resampled['roof_type']= X_resampled['roof_type'].astype('object')
X_resampled['ground_floor_type']= X_resampled['ground_floor_type'].astype('object')
X_resampled['other_floor_type']= X_resampled['other_floor_type'].astype('object')
X_resampled['position']= X_resampled['position'].astype('object')
X_resampled['plan_configuration']= X_resampled['plan_configuration'].astype('object')
X_resampled['legal_ownership_status']= X_resampled['legal_ownership_status'].astype('object')

X_resampled.dtypes

geo_level_1_id                            object
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_cement_mortar_brick     int64
has_superstructure_timber                  int64
has_superstructure_bamboo                  int64
has_superstructure_rc_non_engineered       int64
has_superstructure_rc_engineered           int64
has_superstructure_other                   int64
count_families                             int64
has_secondary_use                          int64
has_secondary_use_agriculture              int64
has_secondary_use_hotel                    int64
has_secondary_use_re

# Splitting into Test and Train Data

In [40]:
X_train, X_val, Y_train, Y_val = train_test_split(train_values,train_labels, test_size = 0.20, random_state = 42)

In [41]:
X_val.head()

,geo_level_1_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
1023112,10,1,20,7,3,t,r,n,f,j,...,0,0,0,0,0,0,0,0,0,0
64407,22,2,60,7,6,t,r,n,f,q,...,0,0,0,0,0,0,0,0,0,0
967834,26,2,50,6,5,t,r,n,v,q,...,0,0,0,0,0,0,0,0,0,0
36669,8,3,50,7,6,o,r,q,f,q,...,0,0,0,0,0,0,0,0,0,0
242842,6,2,15,8,4,t,r,n,f,x,...,0,0,0,0,0,0,0,0,0,0


In [42]:

embed_cols=[i for i in X_train.select_dtypes(include=['object'])]

for i in embed_cols:
    print(i,X_train[i].nunique())

geo_level_1_id 31
land_surface_condition 3
foundation_type 5
roof_type 3
ground_floor_type 5
other_floor_type 4
position 4
plan_configuration 10
legal_ownership_status 4


In [43]:
embed_cols=[i for i in X_train.select_dtypes(include=['object'])]

#converting data to list format to match the network structure
def preproc(X_train, X_val, X_test):

    input_list_train = []
    input_list_val = []
    input_list_test = []
    
    #the cols to be embedded: rescaling to range [0, # values)
    for c in embed_cols:
        raw_vals = np.unique(X_train[c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i       
        input_list_train.append(X_train[c].map(val_map).values)
        input_list_val.append(X_val[c].map(val_map).fillna(0).values)
        input_list_test.append(X_test[c].map(val_map).fillna(0).values)
     
    #the rest of the columns
    other_cols = [c for c in X_train.columns if (not c in embed_cols)]
    input_list_train.append(X_train[other_cols].values)
    input_list_val.append(X_val[other_cols].values)
    input_list_test.append(X_test[other_cols].values)
    
    return input_list_train, input_list_val, input_list_test


In [45]:
for categorical_var in X_train.select_dtypes(include=['object']):
    
    cat_emb_name= categorical_var.replace(" ", "")+'_Embedding'
  
    no_of_unique_cat  = X_train[categorical_var].nunique()
    embedding_size = int(min(np.ceil((no_of_unique_cat)/2), 50 ))
  
    print('Categorica Variable:', categorical_var,
        'Unique Categories:', no_of_unique_cat,
        'Embedding Size:', embedding_size)

Categorica Variable: geo_level_1_id Unique Categories: 31 Embedding Size: 16
Categorica Variable: land_surface_condition Unique Categories: 3 Embedding Size: 2
Categorica Variable: foundation_type Unique Categories: 5 Embedding Size: 3
Categorica Variable: roof_type Unique Categories: 3 Embedding Size: 2
Categorica Variable: ground_floor_type Unique Categories: 5 Embedding Size: 3
Categorica Variable: other_floor_type Unique Categories: 4 Embedding Size: 2
Categorica Variable: position Unique Categories: 4 Embedding Size: 2
Categorica Variable: plan_configuration Unique Categories: 10 Embedding Size: 5
Categorica Variable: legal_ownership_status Unique Categories: 4 Embedding Size: 2


In [46]:
for categorical_var in X_train.select_dtypes(include=['object']):
    
    input_name= 'Input_' + categorical_var.replace(" ", "")
    print(input_name)

Input_geo_level_1_id
Input_land_surface_condition
Input_foundation_type
Input_roof_type
Input_ground_floor_type
Input_other_floor_type
Input_position
Input_plan_configuration
Input_legal_ownership_status


# Creating embeddings for each of the categorical features

In [47]:
input_models=[]
output_embeddings=[]
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

for categorical_var in X_train.select_dtypes(include=['object']):
    
    #Name of the categorical variable that will be used in the Keras Embedding layer
    cat_emb_name= categorical_var.replace(" ", "")+'_Embedding'
  
    # Define the embedding_size
    no_of_unique_cat  = X_train[categorical_var].nunique()
    embedding_size = int(min(np.ceil((no_of_unique_cat)/2), 50 ))
  
    #One Embedding Layer for each categorical variable
    input_model = Input(shape=(1,))
    output_model = Embedding(no_of_unique_cat, embedding_size, name=cat_emb_name)(input_model)
    output_model = Reshape(target_shape=(embedding_size,))(output_model)    
  
    #Appending all the categorical inputs
    input_models.append(input_model)
  
    #Appending all the embeddings
    output_embeddings.append(output_model)
  
#Other non-categorical data columns (numerical). 
#I define single another network for the other columns and add them to our models list.
input_numeric = Input(shape=(len(X_train.select_dtypes(include=numerics).columns.tolist()),))
embedding_numeric = Dense(128)(input_numeric) 
input_models.append(input_numeric)
output_embeddings.append(embedding_numeric)

#At the end we concatenate altogther and add other Dense layers
output = Concatenate()(output_embeddings)
output = Dense(1000, kernel_initializer="uniform")(output)
output = Activation('relu')(output)
output= Dropout(0.4)(output)
output = Dense(512, kernel_initializer="uniform")(output)
output = Activation('relu')(output)
output= Dropout(0.3)(output)
output = Dense(4, activation='softmax')(output)

model = Model(inputs=input_models, outputs=output)
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam',metrics=['sparse_categorical_crossentropy'])


In [48]:
X_test = test_values

# Model

In [57]:
class_weights = {1: 1.75,
            2: 1.0,
            3: 3.03}
history  =  model.fit(X_train_list,Y_train,validation_data=(X_val_list,Y_val) , epochs =  5,verbose=2,class_weight = class_weights)

Train on 208480 samples, validate on 52121 samples
Epoch 1/5
 - 167s - loss: 1.1921 - sparse_categorical_crossentropy: 0.8181 - val_loss: 0.7795 - val_sparse_categorical_crossentropy: 0.7795
Epoch 2/5
 - 129s - loss: 1.1886 - sparse_categorical_crossentropy: 0.8167 - val_loss: 0.8136 - val_sparse_categorical_crossentropy: 0.8136
Epoch 3/5
 - 88s - loss: 1.1911 - sparse_categorical_crossentropy: 0.8179 - val_loss: 0.8225 - val_sparse_categorical_crossentropy: 0.8225
Epoch 4/5
 - 93s - loss: 1.1868 - sparse_categorical_crossentropy: 0.8159 - val_loss: 0.7876 - val_sparse_categorical_crossentropy: 0.7876
Epoch 5/5
 - 87s - loss: 1.1820 - sparse_categorical_crossentropy: 0.8122 - val_loss: 0.7754 - val_sparse_categorical_crossentropy: 0.7754


In [58]:
#pred = model.predict(X_val_list)
y_prob = model.predict(X_val_list)   # Get class probability vector for each sample
y_class = y_prob.argmax(axis=-1) 

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.keras.backend.clear_session()
# Save the model
model.save('Ann_Final.h5')

In [ ]:
import keras
new_model = keras.models.load_model('Ann_Final.h5')

In [59]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_val_list, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(Y_val, y_pred_bool))

52121/52121 [==============================] - 5s 94us/step
              precision    recall  f1-score   support

           1       0.61      0.43      0.51      5170
           2       0.73      0.53      0.62     29487
           3       0.52      0.81      0.64     17464

    accuracy                           0.62     52121
   macro avg       0.62      0.59      0.59     52121
weighted avg       0.65      0.62      0.61     52121



In [ ]:
ann_score

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
plt.rcParams["font.family"] = 'DejaVu Sans'

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
class_labels = ['low','medium','high']
cm = metrics.confusion_matrix(Y_val, y_pred_bool)

    
# plot confusin matrix
plt.figure(figsize=(8,8))
plt.grid(b=False)
plot_confusion_matrix(cm, classes=class_labels, normalize=True, title='Normalized confusion matrix', cmap = plt.cm.Blues)
plt.show()

# get classification report
print('-------------------------')
print('| Classifiction Report |')
classification_report = metrics.classification_report(Y_val, y_pred_bool)
print(classification_report)


In [ ]:
print(confusion_matrix(Y_val, y_pred_bool))

In [ ]:
MicroP = (924+25900+7993)/(924+25900+7993+267+15+9456+4211+35+3320)


In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(new_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [53]:
test_pred = model.predict(X_test_list, batch_size=1, verbose=1)


86868/86868 [==============================] - 118s 1ms/step


In [54]:
test_pred_bool = np.argmax(test_pred, axis=1)
submission_format = pd.read_csv('/Users/ShaimaShoukat/Desktop/BDProject/submission_format.csv', index_col='building_id')

In [55]:
my_submission = pd.DataFrame(data=test_pred_bool,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [56]:
my_submission.to_csv('Ann_weights.csv')

In [ ]:
test_pred

In [ ]:
#over sampling 
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')
train_values_sm,train_labels_sm = smote.fit_sample(new_train,train_labels.values.ravel())
print(train_values_sm.shape,train_labels_sm.shape)

In [ ]:
References:
1)https://github.com/mmortazavi/EntityEmbedding-Working_Example/blob/master/EntityEmbedding.ipynb